[View in Colaboratory](https://colab.research.google.com/github/zacqoo/semantic_segmentation_vegetation_colab/blob/master/main_keras_flow.ipynb)

In [0]:
# restart Kernel
!kill -9 -1

**1. Check to see if you are using GPU**

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

**2. Mount your Google Drive and allow Google Colab environment to access the files on your drive.**

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

**3. Start the main code here, import Keras libraries**

In [4]:
# import keras libraries
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


**4. Import .py file into Google Colab environment**

In [0]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    #import numpy as np 
    #import os
    #import skimage.io as io
    #import skimage.transform as trans
    #import numpy as np
    from keras.models import Model
    from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, merge
    from keras.optimizers import Adam
    #from keras.callbacks import ModelCheckpoint, LearningRateScheduler
    #from keras import backend as keras
    
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [0]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        #img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        #img = img / 255
        #mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

def testGenerator(test_path,num_image=4,target_size = (256,256),flag_multi_class = False,as_gray = False):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

def TestGenerator(test_path,num_image=4,target_size = (256,256),flag_multi_class = False):
    for i in range(num_image):
        img = load_img(os.path.join(test_path,"%d.png"%i),target_size)
        img = img_to_array(img)
        img = img / 255
        #img = np.expand_dims(img, axis=0) 
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        #img = np.reshape(img,(1,)+img.shape)
        yield img

def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

**5. Setup path and data augmentation**

In [0]:
path = 'drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/data/train_keras_flow'
path_image = 'images'
path_mask = 'Cera'

#data augmentation
data_gen_args = dict(rescale= 1./255,
                    rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,path,path_image,path_mask,data_gen_args,save_to_dir = None)

In [0]:
# training
model = unet()
model_checkpoint = ModelCheckpoint('unet_keras_flow_cera.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=150,epochs=20,callbacks=[model_checkpoint])

In [0]:
# load weights
model = unet()
model.load_weights('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/unet_keras_flow_cera.hdf5')

In [0]:
# save weights to drive
model.save_weights('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/unet_keras_flow_cera.hdf5')

In [0]:
# test
datagen = ImageDataGenerator(rescale= 1./255) 
img = datagen.flow_from_directory(('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/data/test_set/'),target_size = (256,256),batch_size=1,shuffle = False, color_mode = "rgb") 
results = model.predict_generator(img,4,verbose=1)

In [0]:
# save predictions
saveResult('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/data/test_set/predictions',results)

**6. Process water site images, overlay prediction masks, change white to red**

In [0]:
# WaterSiteImage
datagen = ImageDataGenerator(rescale= 1./255) 
img = datagen.flow_from_directory(('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/WaterSiteImages/Syer2/'),target_size = (256,256),batch_size=1,shuffle = False, color_mode = "rgb") 
results = model.predict_generator(img,4,verbose=1)

In [0]:
# save predictions
saveResult('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/WaterSiteImages/Syer2/pred_masks/',results)

In [0]:
# import library
from PIL import Image
import numpy as np

In [0]:
import glob
image_list = []
image_filename = []
for filename in sorted(glob.glob('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/WaterSiteImages/Syer2/images/*.JPG')):
    im=Image.open(filename)
    image_list.append(im)
    image_filename.append(filename)

In [0]:
image_filename

In [0]:
import glob
mask_list = []
mask_filename = []
for filename in sorted(glob.glob('drive/My Drive/satellite_vegetation_schisto/code/unet_v3_drone/WaterSiteImages/Syer2/pred_masks/*.png')):
    mask=Image.open(filename)
    mask_list.append(mask)
    mask_filename.append(filename)

In [0]:
mask_filename

In [0]:
for (img, mask, filename) in zip(image_list, mask_list, image_filename):
  # resize mask to match drone image size
  new_width  = img.size[0]
  new_height = img.size[1]
  mask_resize = mask.resize((new_width, new_height), Image.ANTIALIAS)
  # change white color to red, in mask
  mask_resize = mask_resize.convert('RGBA')
  data = np.array(mask_resize)   # "data" is a height x width x 4 numpy array
  red, green, blue, alpha = data.T # Temporarily unpack the bands for readability
  # Replace white with red... (leaves alpha values alone...)
  white_areas = (red == 255) & (blue == 255) & (green == 255)
  data[..., :-1][white_areas.T] = (255, 0, 0) # Transpose back needed
  mask_resize_red = Image.fromarray(data)
  # overlay mask on drone image, add transparency
  background = mask_resize_red.convert("RGBA")
  overlay = img.convert("RGBA")
  new_img = Image.blend(background, overlay, 0.85)
  fullpath = os.path.join(filename + '_pred' + '.' + "png")
  print('saving ', filename)
  new_img.save(fullpath)